In [15]:
import re

import joblib
import pandas as pd
from pandarallel import pandarallel
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

import preprocess

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


## Preprocessing

In [16]:
df = pd.read_csv('../../data/jokes.csv')
df.head()

,theme,text,rating
0,pro-sudey,На суде в Стамбуле обвиняемый сказал:\r\n- На...,5
1,pro-sudey,"- Вы продолжаете утверждать, что обвиняемый н...",4
2,pro-sudey,"На суде.\r\n- Итак, когда дело дошло до столкн...",0
3,pro-sudey,Старую леди сбил автомобиль. На суде ее спраши...,4
4,pro-sudey,"Судья говорит:\r\n- Согласно вашей жалобе, об...",2


In [17]:
df['text'] = df['text'].parallel_apply(preprocess.lemmatize) 

In [18]:
df['rating'] = (df['rating'] - df['rating'].min())/(df['rating'].max() - df['rating'].min())*100.

In [19]:
df.head()

,theme,text,rating
0,pro-sudey,суд стамбул обвиняемый свой жертва нападать но...,2.875399
1,pro-sudey,продолжать утверждать обвиняемый назвать дурак...,2.555911
2,pro-sudey,суд итак дело дойти столкновение противник рук...,1.277955
3,pro-sudey,старый леди сбить автомобиль суд спрашивать де...,2.555911
4,pro-sudey,судья говорить согласно вашея жалоба обвиняемы...,1.916933


## Training

In [20]:
X_train, X_test, y_train, y_test = train_test_split(
  df['text'], df['rating'], test_size=0.3, random_state=42
)

In [21]:
baseline_pipeline = Pipeline(steps=[
  ('vectorizer', TfidfVectorizer()),
  ('model', DummyRegressor())
])

baseline_pipeline.fit(X_train, y_train)

baseline_mse = mean_squared_error(
  baseline_pipeline.predict(X_test),
  y_test
)

print(baseline_mse)

1.5304261477697982


In [22]:
lr_pipeline = Pipeline(steps=[
  ('vectorizer', TfidfVectorizer()),
  ('model', LinearRegression())
])

lr_pipeline.fit(X_train, y_train)

lr_mse = mean_squared_error(
  lr_pipeline.predict(X_test),
  y_test
)

print(lr_mse)
joblib.dump(lr_pipeline, '../../models/lr.pkl')

9.508901605305216


['../../models/lr.pkl']

In [24]:
tr_pipeline = Pipeline(steps=[
  ('vectorizer', TfidfVectorizer()),
  ('model', DecisionTreeRegressor())
])

tr_pipeline.fit(X_train, y_train)

tr_mse = mean_squared_error(
  tr_pipeline.predict(X_test),
  y_test
)

print(tr_mse)
joblib.dump(tr_pipeline, '../../models/tr.pkl')

1.1024651589942072


['../../models/tr.pkl']

In [25]:
df['pred_rating'] = tr_pipeline.predict(df['text'])

In [26]:
df

,theme,text,rating,pred_rating
0,pro-sudey,суд стамбул обвиняемый свой жертва нападать но...,2.875399,2.236422
1,pro-sudey,продолжать утверждать обвиняемый назвать дурак...,2.555911,2.555911
2,pro-sudey,суд итак дело дойти столкновение противник рук...,1.277955,1.277955
3,pro-sudey,старый леди сбить автомобиль суд спрашивать де...,2.555911,2.555911
4,pro-sudey,судья говорить согласно вашея жалоба обвиняемы...,1.916933,1.916933
...,...,...,...,...
130199,raznie,збежать медведь зоопарк передать радио написат...,1.277955,1.277955
130200,raznie,разговаривать грузин гоги авас эх авас трахать...,1.277955,1.277955
130201,raznie,спать гений крепкий,1.277955,1.277955
130202,raznie,очередь последнея нея стоить бабка горбатый но...,1.277955,1.277955
